# AgentCore Memory를 사용한 Strands Agents (단기 메모리)


## 소개

이 튜토리얼은 AgentCore **단기 메모리** (Raw events)를 사용하여 Strands agents로 **개인 어시스턴트 agent**를 구축하는 방법을 보여줍니다. Agent는 `get_last_k_turns`를 사용하여 세션의 최근 대화를 기억하고 사용자가 돌아왔을 때 원활하게 대화를 이어갈 수 있습니다.


### 튜토리얼 세부사항

| 정보                | 세부사항                                                                          |
|:--------------------|:---------------------------------------------------------------------------------|
| 튜토리얼 유형       | 단기 대화형                                                                       |
| Agent 유형          | 개인 어시스턴트 Agent                                                             |
| Agentic Framework   | Strands Agents                                                                   |
| LLM model           | Anthropic Claude Haiku 4.5                                                      |
| 튜토리얼 구성요소   | AgentCore 단기 메모리, AgentInitializedEvent 및 MessageAddedEvent hooks          |
| 예제 복잡도         | 초급                                                                              |

학습 내용:
- 대화 연속성을 위한 단기 메모리 사용
- 마지막 K개의 대화 턴 검색
- 실시간 정보를 위한 웹 검색 tool
- 대화 기록으로 agent 초기화

## 아키텍처
<div style="text-align:left">
    <img src="architecture.png" width="65%" />
</div>

## 사전 요구사항

- Python 3.10+
- AgentCore Memory 권한이 있는 AWS 자격 증명
- AgentCore Memory role ARN
- Amazon Bedrock models 액세스 권한

환경 설정을 시작해봅시다!

## 단계 1: 설정 및 Import

In [ ]:
!pip install -qr requirements.txt

In [ ]:
import logging
from datetime import datetime

# Setup
logging.basicConfig(level=logging.INFO)  # 로깅 레벨을 INFO로 설정
logger = logging.getLogger("personal-agent")

In [ ]:
# Imports
import os
from strands import Agent, tool
from strands.hooks import AgentInitializedEvent, HookProvider, HookRegistry, MessageAddedEvent
from bedrock_agentcore.memory import MemoryClient

# Configuration
REGION = os.getenv('AWS_REGION', 'us-west-2') # AWS region for the agent
ACTOR_ID = "user_123" # 고유 식별자 (AgentID, User ID 등)
SESSION_ID = "personal_session_001" # 세션 고유 식별자


## 단계 2: 웹 검색 Tool

먼저 agent를 위한 간단한 웹 검색 tool을 만들어봅시다.

In [ ]:
from ddgs.exceptions import DDGSException, RatelimitException
from ddgs import DDGS

@tool
def websearch(keywords: str, region: str = "us-en", max_results: int = 5) -> str:
    """Search the web for updated information.
    
    Args:
        keywords (str): The search query keywords.
        region (str): The search region: wt-wt, us-en, uk-en, ru-ru, etc..
        max_results (int | None): The maximum number of results to return.
    Returns:
        List of dictionaries with search results.
    
    """
    try:
        # DuckDuckGo API를 사용하여 웹 검색 수행
        results = DDGS().text(keywords, region=region, max_results=max_results)
        return results if results else "No results found."
    except RatelimitException:
        return "Rate limit reached. Please try again later."
    except DDGSException as e:
        return f"Search error: {e}"
    except Exception as e:
        return f"Search error: {str(e)}"

logger.info("✅ Web search tool ready")

## 단계 3: Memory 리소스 생성
단기 메모리의 경우, 전략 없이 memory 리소스를 생성합니다. 이는 `get_last_k_turns`로 검색할 수 있는 원시 대화 턴을 저장합니다.


In [ ]:
from botocore.exceptions import ClientError

# Initialize Memory Client
client = MemoryClient(region_name=REGION)
memory_name = "PersonalAgentMemory"

try:
    # 전략 없이 memory 리소스 생성 (단기 메모리만 사용)
    memory = client.create_memory_and_wait(
        name=memory_name,
        strategies=[],  # 단기 메모리를 위한 전략 없음
        description="Short-term memory for personal agent",
        event_expiry_days=7, # 단기 메모리 보관 기간 (최대 365일)
    )
    memory_id = memory['id']
    logger.info(f"✅ Created memory: {memory_id}")
except ClientError as e:
    logger.info(f"❌ ERROR: {e}")
    # 이미 존재하는 memory인 경우 기존 ID 조회
    if e.response['Error']['Code'] == 'ValidationException' and "already exists" in str(e):
        memories = client.list_memories()
        memory_id = next((m['id'] for m in memories if m['id'].startswith(memory_name)), None)
        logger.info(f"Memory already exists. Using existing memory ID: {memory_id}")
except Exception as e:
    # memory 생성 중 오류 발생 시 표시
    logger.error(f"❌ ERROR: {e}")
    import traceback
    traceback.print_exc()
    # 오류 발생 시 정리 - 부분적으로 생성된 memory 삭제
    if memory_id:
        try:
            client.delete_memory_and_wait(memory_id=memory_id)
            logger.info(f"Cleaned up memory: {memory_id}")
        except Exception as cleanup_error:
            logger.error(f"Failed to clean up memory: {cleanup_error}")

## 단계 4: Memory Hook

이 단계에서는 memory 작업을 자동화하는 커스텀 `MemoryHookProvider` 클래스를 정의합니다. Hook은 agent의 실행 생명주기에서 특정 시점에 실행되는 특수 함수입니다. 우리가 만드는 memory hook은 두 가지 주요 기능을 수행합니다:
1. **최근 대화 로드**: `AgentInitializedEvent` hook을 사용하여 agent가 초기화될 때 최근 대화 기록을 자동으로 로드합니다.
2. **마지막 메시지 저장**: 새로운 대화 메시지를 저장합니다.

이를 통해 수동 관리 없이 원활한 메모리 경험을 제공합니다.

In [ ]:
class MemoryHookProvider(HookProvider):
    def __init__(self, memory_client: MemoryClient, memory_id: str):
        self.memory_client = memory_client
        self.memory_id = memory_id
    
    def on_agent_initialized(self, event: AgentInitializedEvent):
        """Agent 시작 시 최근 대화 기록 로드"""
        try:
            # Agent state에서 세션 정보 가져오기
            actor_id = event.agent.state.get("actor_id")
            session_id = event.agent.state.get("session_id")
            
            if not actor_id or not session_id:
                logger.warning("Missing actor_id or session_id in agent state")
                return
            
            # Memory에서 최근 5개의 대화 턴 로드
            recent_turns = self.memory_client.get_last_k_turns(
                memory_id=self.memory_id,
                actor_id=actor_id,
                session_id=session_id,
                k=5
            )
            
            if recent_turns:
                # 컨텍스트를 위한 대화 기록 포맷팅
                context_messages = []
                for turn in recent_turns:
                    for message in turn:
                        role = message['role']
                        content = message['content']['text']
                        context_messages.append(f"{role}: {content}")
                
                context = "\n".join(context_messages)
                # Agent의 system prompt에 컨텍스트 추가
                event.agent.system_prompt += f"\n\nRecent conversation:\n{context}"
                logger.info(f"✅ Loaded {len(recent_turns)} conversation turns")
                
        except Exception as e:
            logger.error(f"Memory load error: {e}")
    
    def on_message_added(self, event: MessageAddedEvent):
        """메시지를 memory에 저장"""
        messages = event.agent.messages
        try:
            # Agent state에서 세션 정보 가져오기
            actor_id = event.agent.state.get("actor_id")
            session_id = event.agent.state.get("session_id")

            # 마지막 메시지에 텍스트가 있는 경우 저장
            if messages[-1]["content"][0].get("text"):
                self.memory_client.create_event(
                    memory_id=self.memory_id,
                    actor_id=actor_id,
                    session_id=session_id,
                    messages=[(messages[-1]["content"][0]["text"], messages[-1]["role"])]
                )
        except Exception as e:
            logger.error(f"Memory save error: {e}")
    
    def register_hooks(self, registry: HookRegistry):
        # Memory hook 등록
        registry.add_callback(MessageAddedEvent, self.on_message_added)
        registry.add_callback(AgentInitializedEvent, self.on_agent_initialized)

## 단계 5: 웹 검색 기능을 가진 개인 어시스턴트 Agent 생성

In [ ]:
def create_personal_agent():
    """Memory와 웹 검색 기능을 가진 개인 어시스턴트 agent 생성"""
    agent = Agent(
        name="PersonalAssistant",
        model="global.anthropic.claude-haiku-4-5-20251001-v1:0",  # or your preferred model
        system_prompt=f"""You are a helpful personal assistant with web search capabilities.
        
        You can help with:
        - General questions and information lookup
        - Web searches for current information
        - Personal task management
        
        When you need current information, use the websearch function.
        Today's date: {datetime.today().strftime('%Y-%m-%d')}
        Be friendly and professional.""",
        hooks=[MemoryHookProvider(client, memory_id)],  # Memory hook 등록
        tools=[websearch],  # 웹 검색 tool 등록
        state={"actor_id": ACTOR_ID, "session_id": SESSION_ID}  # 세션 정보 저장
    )
    return agent

# Create agent
agent = create_personal_agent()
logger.info("✅ Personal agent created with memory and web search")

#### 축하합니다! Agent가 준비되었습니다! :) 
## Agent 테스트하기

In [ ]:
# Test conversation with memory
print("=== First Conversation ===")
print(f"User: My name is Alex and I'm interested in learning about AI.")
print(f"Agent: ", end="")
agent("My name is Alex and I'm interested in learning about AI.")

In [ ]:
print(f"User: Can you search for the latest AI trends in 2025?")
print(f"Agent: ", end="")
agent("Can you search for the latest AI trends in 2025?")

In [ ]:
print(f"User: I'm particularly interested in machine learning applications.")
print(f"Agent: ", end="")
agent("I'm particularly interested in machine learning applications.")

## 메모리 연속성 테스트

메모리 시스템이 올바르게 작동하는지 테스트하기 위해 agent의 새 인스턴스를 생성하고 이전에 저장된 정보에 액세스할 수 있는지 확인합니다:

In [ ]:
# Create new agent instance (simulates user returning)
print("=== User Returns - New Session ===")
new_agent = create_personal_agent()  # 새 agent 인스턴스 생성 (사용자 복귀 시뮬레이션)

# Test memory continuity
print(f"User: What was my name again?")
print(f"Agent: ", end="")
new_agent("What was my name again?")

print(f"User: Can you search for more information about machine learning?")
print(f"Agent: ", end="")
new_agent("Can you search for more information about machine learning?")

## 저장된 Memory 확인

In [ ]:
# Check what's stored in memory
print("=== Memory Contents ===")
recent_turns = client.get_last_k_turns(
    memory_id=memory_id,
    actor_id=ACTOR_ID,
    session_id=SESSION_ID,
    k=3 # k 값을 조정하여 더 많거나 적은 턴 확인 가능
)

for i, turn in enumerate(recent_turns, 1):
    print(f"Turn {i}:")
    for message in turn:
        role = message['role']
        # 긴 메시지는 100자로 제한하여 표시
        content = message['content']['text'][:100] + "..." if len(message['content']['text']) > 100 else message['content']['text']
        print(f"  {role}: {content}")
    print()

## 요약

이 튜토리얼은 개인 어시스턴트 agent를 구축하는 방법을 보여주었습니다. 학습한 내용:

- 전략 없이 memory 리소스 생성
- 대화 기록을 위한 `get_last_k_turns` 사용
- Agent에 웹 검색 기능 추가
- 컨텍스트 로딩을 위한 memory hook 구현

**다음 단계:**
- 더 정교한 tool 추가
- 장기 메모리 전략 구현
- 여러 소스를 사용한 검색 기능 향상

## 정리 (선택사항)

In [ ]:
# Uncomment to delete memory resource
# Memory 리소스를 삭제하려면 아래 주석 해제
# client.delete_memory_and_wait(memory_id)
# logger.info(f"✅ Deleted memory: {memory_id}")